# CODEATHON 3: Recognizing UVA landmarks with neural nets (50 pts)
![UVA Grounds](http://faculty.virginia.edu/lazzara/images/UVAgrounds.jpg) 

The UVA Grounds is known for its Jeffersonian architecture and place in U.S. history as a model for college and university campuses throughout the country. Throughout its history, the University of Virginia has won praise for its unique Jeffersonian architecture. 

In this codeathon, you will attempt the build an image recognition system to classify different buildlings/landmarks on Grounds. Your will earn 50 points for this codeathon plus 5 bonus points. Part of the total grade (10 pts) is for contributing 100+ images to the dataset, and the 5 bonus pts are for contributing 200+ images to the dataset.

To make it easier for you, some codes have been provided to help you process the data, you may modify it to fit your need. You must submit the .ipynb file via UVA Collab with the following format: yourcomputingID_codeathon_3.ipynb

Thank you for collecting part of the dataset, best of luck, and have fun! 

# Load Packages

In [1]:
import sys
import sklearn
import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from functools import partial

%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras

np.random.seed(49)
tf.random.set_seed(49)

TensorFlow 2.x selected.


# Import Dataset
The full dataset is huge (+37GB) with +13K images. So it will take a while to download, extract, and process. To save you time and effort, a subset of the data has been extracted to Dropbox to containly only 5 classes and 10 images each. 

Later, we will share a lite version of the data via another dropbox URL. This dataset will contain all 18 classes, and 100 images each. This dataset will be the one you will benchmark for your grade. If you are up for a challenge (and perhaps bonus points), contact the instructor for the full dataset!

In [2]:
# Download dataset from Dropbox
!wget "https://uvalandmark18.s3.amazonaws.com/UVALandmark18.zip"

--2019-11-28 13:20:23--  https://uvalandmark18.s3.amazonaws.com/UVALandmark18.zip
Resolving uvalandmark18.s3.amazonaws.com (uvalandmark18.s3.amazonaws.com)... 52.216.112.219
Connecting to uvalandmark18.s3.amazonaws.com (uvalandmark18.s3.amazonaws.com)|52.216.112.219|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6194605880 (5.8G) [application/zip]
Saving to: ‘UVALandmark18.zip’

UVALandmark18.zip   100%[===================>]   5.77G  16.3MB/s    in 6m 9s   

2019-11-28 13:26:33 (16.0 MB/s) - ‘UVALandmark18.zip’ saved [6194605880/6194605880]



In [3]:
# Extract content
!unzip "/content/UVALandmark18.zip"

Archive:  /content/UVALandmark18.zip
warning [/content/UVALandmark18.zip]:  4294967296 extra bytes at beginning or within zipfile
  (attempting to process anyway)
file #1:  bad zipfile offset (local header sig):  4294967296
  (attempting to re-compensate)
   creating: UVALandmark18/
   creating: UVALandmark18/Academical Village/
  inflating: UVALandmark18/Academical Village/ks5qug:1574201282238  
  inflating: UVALandmark18/Academical Village/Mec2wr:1574176804076  
  inflating: UVALandmark18/Academical Village/jw6qs:1574027372543  
  inflating: UVALandmark18/Academical Village/jw6qs:1574027360247  
  inflating: UVALandmark18/Academical Village/Mec2wr:1574176408847  
  inflating: UVALandmark18/Academical Village/asz9qm:1574276428474  
  inflating: UVALandmark18/Academical Village/asz9qm:1574276517718  
  inflating: UVALandmark18/Academical Village/ANON:1574177114794  
  inflating: UVALandmark18/Academical Village/Mec2wr:1574176921561  
  inflating: UVALandmark18/Academical Village/.DS_St

In [0]:
import os
PATH = "/content/UVALandmark18"
for folder in os.listdir(PATH):
    ds_store = PATH+"/"+folder+"/.DS_Store"
    if os.path.exists(ds_store):
        os.remove(ds_store)

In [5]:
from sklearn.datasets import load_files 
from keras.utils import np_utils

# define function to load train, test, and validation datasets
def load_dataset(path, num_classes):
    #Load text files with categories as subfolder names.
    data = load_files(path)
    filenames = np.array(data['filenames'])
    targets = np_utils.to_categorical(np.array(data['target']), num_classes)
    return filenames, targets

n_classes = 18;
# Make sure you create the class names that match the order of their appearances in the "files" variable
#class_names = ["Chapel", "Olsson", "Rice", "Rotunda", "Scott"]
# load train, test, and validation datasets
files, targets = load_dataset('/content/UVALandmark18/',n_classes)
print(type(files), files)   

Using TensorFlow backend.


<class 'numpy.ndarray'> ['/content/UVALandmark18/Bavaro Hall/csn5aw:1573933936836'
 '/content/UVALandmark18/Brooks Hall/xy3jh :1574189685070'
 '/content/UVALandmark18/Olsson Hall/yl9gq:1574017354270' ...
 '/content/UVALandmark18/Madison Hall/ks5qug:1574201675326'
 '/content/UVALandmark18/Minor Hall/dbl3jf :1574194956314'
 '/content/UVALandmark18/Rice Hall/Jwh6ry :1574275138938']


In [0]:
from sklearn.model_selection import train_test_split
# Split to train-validate-test sets. DO NOT CHANGE THE TEST RATIO OR RANDOM STATE
train_files, test_files, train_targets, test_targets = train_test_split(files, targets, test_size=0.3, random_state=49)
train_files, val_files, train_targets, val_targets = train_test_split(train_files, train_targets, test_size=0.2, random_state=49)

In [7]:
from keras.preprocessing import image
from tqdm import tqdm # progress bar

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)           

# pre-process the data for Keras - Converts to (224, 224) and converts into a numpy array using PIL.
# NOTE: This code does not include any data augmentation, but you can modify it to include the augmentation operation.
X_train = paths_to_tensor(train_files).astype('float32')/255
X_val = paths_to_tensor(val_files).astype('float32')/255
X_test = paths_to_tensor(test_files).astype('float32')/255
# Convert y_targets into labels
y_train = np.asarray([np.where(r==1)[0][0] for r in train_targets], dtype=np.uint8)
y_test = np.asarray([np.where(r==1)[0][0] for r in test_targets], dtype=np.uint8)
y_val = np.asarray([np.where(r==1)[0][0] for r in val_targets], dtype=np.uint8)

100%|██████████| 642/642 [02:12<00:00,  5.46it/s]


In [8]:
X_train.shape

(1196, 224, 224, 3)

# It's your turn: Building a classifier for UVA Landmark Dataset
You may design your own architecture OR re-use any of the exising frameworks (recommended). 

Best of luck!

In [11]:
base_model=keras.applications.xception.Xception(weights="imagenet", include_top=False)
pooling=keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(18, activation='softmax')(pooling)
model=keras.Model(inputs=base_model.input, outputs=output)

for layer in base_model.layers:
  layer.trainable=True
optimizer=keras.optimizers.SGD(lr=0.01, momentum = 0.9, nesterov=True)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=15, validation_data=[X_val, y_val])

Train on 1196 samples, validate on 300 samples
Epoch 1/15
1196/1196 [==============================] - 54s 45ms/sample - loss: 2.2788 - accuracy: 0.3294 - val_loss: 2.0471 - val_accuracy: 0.3600
Epoch 2/15
1196/1196 [==============================] - 47s 39ms/sample - loss: 0.7519 - accuracy: 0.8662 - val_loss: 1.2017 - val_accuracy: 0.6033
Epoch 3/15
1196/1196 [==============================] - 47s 39ms/sample - loss: 0.1913 - accuracy: 0.9699 - val_loss: 0.6164 - val_accuracy: 0.8000
Epoch 4/15
1196/1196 [==============================] - 47s 39ms/sample - loss: 0.0612 - accuracy: 0.9950 - val_loss: 0.5039 - val_accuracy: 0.8533
Epoch 5/15
1196/1196 [==============================] - 47s 39ms/sample - loss: 0.0282 - accuracy: 0.9992 - val_loss: 0.4452 - val_accuracy: 0.8667
Epoch 6/15
1196/1196 [==============================] - 47s 39ms/sample - loss: 0.0181 - accuracy: 1.0000 - val_loss: 0.4330 - val_accuracy: 0.8767
Epoch 7/15
1196/1196 [==============================] - 47s 39ms/

In [12]:
#Evaluate the model. 
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)

642/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

As shown above, I applied Transfer Learning by utilizing lower layers from Xception and connecting with own pooling layer and dense layer to classify UVA Landmark images to 18 classes. I freezed up the pre-trained lower layers and only trained my own added layers first. Then I trained the whole network with a lower learning rate and achieved the final accuracy 0.917.